In [2]:
import pydeequ
from pyspark.sql import SparkSession


spark_configs = {
    'spark.master': 'spark://spark-iceberg:7077',
    'spark.sql.catalog.airline': 'org.apache.iceberg.spark.SparkCatalog',
    # 'spark.sql.catalog.airline': 'org.apache.iceberg.spark.SparkSessionCatalog',
    'spark.sql.catalog.airline.io-impl': 'org.apache.iceberg.aws.s3.S3FileIO',
    'spark.sql.catalog.airline.s3.endpoint': 'http://minio:9000',
    'spark.sql.catalog.airline.type': 'rest',
    'spark.sql.catalog.airline.uri': 'http://rest:8181',
    'spark.sql.catalog.airline.warehouse': 's3://warehouse',
    'spark.sql.defaultCatalog': 'airline',
    'spark.driver.memory': '1G',
    'spark.executor.memory': '1G'
}

# Initialize SparkSession
spark = (
    SparkSession
    .builder
    .appName('Data Quality Testing')
    .config(map=spark_configs)
    .getOrCreate()
)

24/12/31 15:33:21 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [ ]:
from 